In [3]:
import os
from pathlib import Path
from collections import namedtuple
from operator import add, mul, floordiv as div, eq as eql, mod, sub


In [4]:
FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day24.txt'

In [5]:
instructions = []

# break up the instructions into the 14 functions
# each one is a function of `w` and the registers

with open(FOLDER / in_file) as f:
    for line in f:
        op, *args = line.split()
        if op == 'inp':
            instructions.append([])
            continue
        args = [int(a) if a.lstrip('-').isdigit() else a for a in args]
        instructions[-1].append((op, args))

# the instructions are the same except:
for a,b in zip(instructions[4], instructions[11]):
    arrow =  "" if a==b else "<===="
    print(f"{str(a):<20}  {str(b):<20} {arrow}")

('mul', ['x', 0])     ('mul', ['x', 0])    
('add', ['x', 'z'])   ('add', ['x', 'z'])  
('mod', ['x', 26])    ('mod', ['x', 26])   
('div', ['z', 1])     ('div', ['z', 26])   <====
('add', ['x', 15])    ('add', ['x', -1])   <====
('eql', ['x', 'w'])   ('eql', ['x', 'w'])  
('eql', ['x', 0])     ('eql', ['x', 0])    
('mul', ['y', 0])     ('mul', ['y', 0])    
('add', ['y', 25])    ('add', ['y', 25])   
('mul', ['y', 'x'])   ('mul', ['y', 'x'])  
('add', ['y', 1])     ('add', ['y', 1])    
('mul', ['z', 'y'])   ('mul', ['z', 'y'])  
('mul', ['y', 0])     ('mul', ['y', 0])    
('add', ['y', 'w'])   ('add', ['y', 'w'])  
('add', ['y', 2])     ('add', ['y', 7])    <====
('mul', ['y', 'x'])   ('mul', ['y', 'x'])  
('add', ['z', 'y'])   ('add', ['z', 'y'])  


For the three different 'parameters' that change from instruction only the second two seem to make a difference in out imput.
 

In [6]:
# each set of the 14 instruction sets is the same except for
# three parameters parameters. We don't need the first one for this

params = namedtuple('Params', list('ab'), defaults=[0, 0])
functionParams = []

for instruction_set in instructions:
    p = params(instruction_set[4][1][1],instruction_set[14][1][1])
    functionParams.append(p)

functionParams

[Params(a=11, b=8),
 Params(a=12, b=8),
 Params(a=10, b=12),
 Params(a=-8, b=10),
 Params(a=15, b=2),
 Params(a=15, b=8),
 Params(a=-11, b=4),
 Params(a=10, b=9),
 Params(a=-3, b=10),
 Params(a=15, b=3),
 Params(a=-3, b=7),
 Params(a=-1, b=7),
 Params(a=-10, b=2),
 Params(a=-16, b=2)]

#### Part One

In [7]:
stack = []
digits = [None] * 14

for i, params in enumerate(functionParams):

    if params.a > 9:
        stack.append((i, params.b))
    else:
        last_i, last_b = stack.pop()
        interval = last_b + params.a      
        possible = filter(lambda digit: digit + interval < 10, range(1, 10))
        last_digit = max(possible)
        
        digits[last_i] = last_digit
        digits[i] = last_digit + interval

s = ''.join(map(str, digits))

print("Part one: ", s) 

Part one:  99598963999971


### Part Two
Basically the same, but take the min

In [8]:
stack = []
digits = [None] * 14

for i, params in enumerate(functionParams):

    if params.a > 9:
        stack.append((i, params.b))
    else:
        last_i, last_b = stack.pop()
        interval = last_b + params.a       
        ### need to add zero as lower bound
        possible = filter(lambda digit: 0 < digit + interval < 10, range(1, 10))
        ### now just take the min
        last_digit = min(possible)
        digits[last_i] = last_digit
        digits[i] = last_digit + interval

s = ''.join(map(str, digits))

print("Part Two: ", s) 

Part Two:  93151411711211


### Less-than-useful work, but makes a nice validator

In [9]:
Register = namedtuple("Register", list('wxyz'), defaults=[0,0,0,0]) 

def dec(f):
    def inst(registers, *args):
        register = args[0]
        args = [a if isinstance(a, int) else getattr(registers,a) for a in args] 
        return registers._replace(**{register: f(*args)})
    return inst

# the decorated will create functions that
# accept  registers and args arguments
operators = {
        'add': dec(add),
        'mul': dec(mul),
        'div': dec(div),
        'eql': dec(eql),
        'mod': dec(mod),
}


# create 14 functions from those instructions:
# registers are represented as a tuple
# (w, x, y, z)

def make_function(instuction_set):
    def f(w, reg):
        reg = reg._replace(w=w)
        for op, args in instuction_set:
            reg = operators[op](reg, *args)
        return reg
    return f


registers = Register()
funcs = [make_function(i) for i in instructions]


In [10]:
# compose all 14 functions for a validator:
def validate(data):
    registers = Register()
    for f, w in zip(funcs, data):
        registers = f(w, registers)
  
    return registers
    
part_one = 99598963999971
part_two = 93151411711211

part_one = map(int, str(part_one))
part_two = map(int, str(part_two))

print(validate(part_one))
print(validate(part_two))


Register(w=1, x=False, y=0, z=0)
Register(w=1, x=False, y=0, z=0)
